Computational Linguistics &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; <br>Biao Yun

<center> 

# HMM & CRF
### 2023.04.01

</center>

#### 引入所需 packages

In [1]:
from __future__ import division
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import random

## (1) Change the search algorithm of a HMM model into Viterbi algorithm. (10%)

* 這邊使用老師範例原本的例子，改成用 viterbi 來訓練 HMM 模型
* HMM（隱馬可夫模型）包含以下幾個要素：
    1. 狀態空間（State Space）：HMM 模型中的每個時間點都對應到一個隱含的狀態(hidden state)，這些狀態組成了一個狀態空間。
    2. 觀察序列（Observation Sequence）：對於每個時間點，會有其相對應可觀察到的符號或特徵，這些符號組成了一個觀察序列。
    3. 轉移機率（Transition Probability）：HMM 模型中，每個狀態都有一個轉移到另一個狀態的機率（hidden states 之間相互轉換的機率），這些轉移機率構成了一個狀態轉移矩陣。
    4. 發射機率（Emission Probability）：HMM 模型中，每個狀態都有一個將該狀態轉換成某個觀察值的機率（從 hidden states 發射成 observed states 的機率），這些機率構成了一個發射矩陣。
    5. 初始機率（Initial Probability）：HMM 模型中，每個狀態的起始機率，也就是模型在第一個時間點時處於每個狀態的機率。
* 故以下建構 HMM 模型時，會依照上述之邏輯建構與訓練
* 在 HMM 中，狀態序列是隱藏的，而觀察序列是可見的。我們需要找到最可能的狀態序列，以便對觀察序列進行分類、標記等操作。Viterbi 算法通過動態規劃來計算所有可能的狀態序列中最可能的一條最佳路徑，因此能夠大大提升模型的計算效率。

#### 首先定義 hidden states

In [2]:
# define the hidden states
hidden_states = ["Rainy", "Sunny"]
n_states = len(hidden_states)


#### 接著定義 observed states

In [3]:
# define the observed states (words)
observed_types = ["walk", "shop", "clean"]
n_observed_types = len(observed_types)

#### 定義 initial states

In [4]:
# define the initial state probabilities
start_probability = np.array([0.6, 0.4])

#### 定義轉移矩陣

In [5]:
# define the transition probabilities
transition_probability = np.array([
  [0.7, 0.3],
  [0.4, 0.6]
])

#### 定義發射矩陣

In [6]:
# define the emission probabilities
emission_probability = np.array([
  [0.1, 0.4, 0.5],
  [0.6, 0.3, 0.1]
])

#### 開始訓練模型

In [7]:
# build and train the HMM model
model = hmm.CategoricalHMM(n_components=n_states)
model.startprob = start_probability
model.transmat = transition_probability
model.emissionprob = emission_probability

#### 印出預測結果和每一個 states 的 log prabability

In [8]:
# predict a sequence of hidden states based on visible states
real_observation = np.array([[0, 2, 1, 1, 2, 0, 2]]).T
model = model.fit(real_observation)
logprob, output_hidden = model.decode(real_observation, algorithm='viterbi')
print("Real Observation:", ", ".join(map(lambda x: observed_types[x], real_observation.T[0])))
print("Output of hidden states:", ", ".join(map(lambda x: hidden_states[x], output_hidden)))

# print log probability of each state
logprobs = model.score_samples(real_observation)[1]
for i, state in enumerate(hidden_states):
    rounded_probs = [round(prob, 5) for prob in logprobs[:, i]]
    print("Log probability of state", state, ":", ", ".join(map(str, rounded_probs)))

Real Observation: walk, clean, shop, shop, clean, walk, clean
Output of hidden states: Rainy, Sunny, Rainy, Sunny, Sunny, Rainy, Sunny
Log probability of state Rainy : 1.0, 0.0, 0.45175, 0.45175, 0.0, 1.0, 0.0
Log probability of state Sunny : 0.0, 1.0, 0.54825, 0.54825, 1.0, 0.0, 1.0


## 第一題 Reflection

我去查 HMM 的 document 時才發現，原來他預設的 algorithm 就是 Viterbi，所以其實不需要特別去調就會是 Viterbi 算法了！
另外，在印出每一個 state 的 log probability 的時候發現因為 log probability 是 float 型態，所以沒辦法用 lambda 的方式完成，只能用 enumerate 來抓到它的 index，一個一個去印出來。除此之外，log probability 的數值小數點太多了，這邊為了方便閱讀，只取到小數點後第五位，在盡量不偏離原本數值太多的情況下，保證其易讀性。

關於不同的 HMM 模型：

CategoricalHMM、GaussianHMM 和 GMMHMM 都是 HMM 模型的變種，它們之間的主要差別在於對於隱藏狀態和觀測狀態的建模方式不同。

1. CategoricalHMM：假設觀測狀態為類別型變數，即每個觀測狀態是由一個類別標籤表示的。因此，CategoricalHMM 使用的是類別分佈來表示觀測狀態的生成方式。每個隱藏狀態對應到一個類別分佈，表示在該狀態下生成觀測狀態的機率分佈。
2. GaussianHMM：假設觀測狀態為連續型變數，即每個觀測狀態是由一個實數向量表示的。因此，GaussianHMM 使用的是高斯分佈來表示觀測狀態的生成方式。每個隱藏狀態對應到一個高斯分佈，表示在該狀態下生成觀測狀態的機率分佈。
3. GMMHMM：GMMHMM 是 GaussianHMM 的變種，假設每個隱藏狀態下的觀測狀態是由多個高斯分佈混合而成的。因此，每個隱藏狀態對應到一個高斯混合模型，其中每個高斯分佈表示在該狀態下生成觀測狀態的機率分佈，高斯分佈的參數（如均值和方差）是由 EM 算法學習得到的。

總之，雖然看完我還是覺得有點抽象，但重點就是 CategoricalHMM 假設觀測狀態是類別型變數，GaussianHMM 假設觀測狀態是連續型變數，而 GMMHMM 假設每個隱藏狀態下的觀測狀態是由多個高斯分佈混合而成的。所以要根據自己的假設來決定要用哪一種 HMM 模型，畢竟這會影響 HMM 模型的效果。

## (2) Make yourself an example that can use a HMM model to deal with a language issue (e.g., POS tagging or others). Draw a HMM model, and train the model with output results. (40%)  *note that you'll need to attach your HMM figure with an open shared link in the markdown in order to show it. 


#### 假設我們有以下一組觀察結果及其相應的 POS 標籤

| Observation | POS Tag |
| :-----: | :----: |
| I | PRON |
| am | VERB |
| happy | ADJ |
| to | PART |
| learn | VERB |
| about | ADP |
| HMM| NOUN |
| models | NOUN |

* 做 POS taggin tasks 時，對 HMM 來說，我們想知道的 POS 標籤就是 hidden states，而我們所看到的句子就是 observation states
* 但由於這句話的 POS tags 有點多有點複雜，這邊僅舉簡單的 HMM 例子，故將這些 POS tags 簡化（即簡化 hidden states）
* 留下名詞 NOUN、動詞 VERB 和不屬於這兩類的其他 Other，表示如下：
    1. Noun (N)
    2. Verb (V)
    3. Other (O)

#### 如下圖，該模型有 3 個 hidden states，每個 hidden states 的 emission probabilities 以虛線表示，hidden states 之間的 transition probabilities 則以實線表示，圖例位於圖片右上角。

<img src="https://img.onl/bURZX5" width="100%">

#### 接下來，我們將上圖的 HMM 寫成 codes

* 這邊會以 CategoricalHMM 來做訓練
* 在自然語言處理中，POS tagging 任務屬於類別型變數。因為在詞性標註中，每個單詞都被標記為特定的詞性，這些詞性通常是一個固定的詞性集合，而不是一個連續的範圍。因此，POS tagging 的輸出就會是一個固定的類別集合，而不是一個連續的值範圍。

#### 首先定義 hidden states

In [9]:
# define the hidden states
new_hidden_states = ["Noun", "Verb", "Other"]
new_n_states = len(new_hidden_states)

#### 接著定義 observed states

In [10]:
# define the observed states (words)
new_observed_types = ["I", "am", "happy", "to", "learn", "about", "HMM", "models"]
new_n_observed_types = len(new_observed_types)

#### 定義 initial states

In [11]:
# define the initial state probabilities
new_start_probability = np.array([0.4, 0.3, 0.3])

#### 定義轉移矩陣

In [12]:
# define the transition probabilities
new_transition_probability = np.array([
  [0.5, 0.3, 0.2],
  [0.3, 0.4, 0.3],
  [0.2, 0.3, 0.5]
])

#### 定義發射矩陣

In [13]:
# define the emission probabilities
new_emission_probability = np.array([
  [0.05, 0.05, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.2, 0.05, 0.2, 0.2, 0.1, 0.0, 0.25],
  [0.25, 0.05, 0.1, 0.25, 0.0, 0.2, 0.05, 0.1]
])

#### 開始訓練模型

In [14]:
# build and train the HMM model
new_model = hmm.CategoricalHMM(n_components=new_n_states, n_iter=1000, init_params=[])
new_model.startprob_ = new_start_probability
new_model.transmat_ = new_transition_probability
new_model.emissionprob_ = new_emission_probability

#### 印出預測結果和每一個 states 的 log prabability

In [15]:
# predict a sequence of hidden states based on visible states
X = np.array([
    [0, 1, 2, 3, 4, 5, 6, 7]
]).T
new_model.fit(X)
Y = new_model.predict(X)
print("Observations:", ", ".join(new_observed_types))
print("Predicted POS Tags:", ", ".join(map(lambda x: new_hidden_states[x], Y)))

# print the log probabilities of each hidden state for the given sequence
new_real_observation = np.array([[0, 1, 2, 3, 4, 5, 6, 7]]).T
new_logprobs = new_model.score_samples(new_real_observation)[1]
for s, stat in enumerate(new_hidden_states):
    new_rounded_probs = [round(proba, 5) for proba in new_logprobs[:, s]]
    print("Log probability of state", stat, ":", ", ".join(map(str, new_rounded_probs)))

Fitting a model with 29 free scalar parameters with only 8 data points will result in a degenerate solution.


Observations: I, am, happy, to, learn, about, HMM, models
Predicted POS Tags: Noun, Verb, Noun, Verb, Verb, Other, Other, Other
Log probability of state Noun : 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0
Log probability of state Verb : 0.0, 1.0, 0.0, 1.0, 1.0, 1e-05, 0.0, 0.0
Log probability of state Other : 0.0, 0.0, 0.0, 0.0, 0.0, 0.99999, 1.0, 1.0


## 第二題 Reflection

這邊發現跳出了 warning，警告資料太少但模型需要擬合的參數太多，所以模型可能沒辦法有好的表現的問題。其實從結果也看得出來，當簡化 tags 之後，整句話的 "Other" 變得很多，但又沒有足夠的資料去訓練，導致模型很容易沒辦法辨別 Other 和其他標籤的區別，形成混亂。且 Pronoun 其實也帶有 Noun 的特性，所以模型將 I 分成 Noun 其實情有可原。不過 HMM、models 都變成 Other 的部分就像我前面提到的，硬要去區分且對比的資訊不夠多時就很容易發生模型表現不好的情況。

## (3) Use the development set to help the model find the best parameters, and then evaluate the model on the test set. You can choose either Spanish or Dutch dataset. (50%)

* 以下使用 nltk conll2002 資料集
* nltk conll2002 資料集是一個用於命名實體識別 Named Entity Recognition (NER) 和詞性標註 Part-Of-Speech Tagging (POS) 的資料集，包含荷蘭文和西班牙文。其中包括人名、組織機構、位置等不同類型的實體識別。 Conll2002 是 NLP 領域中的一個重要資源，可以用於訓練和評估自然語言處理模型。
* 此次訓練將使用 nltk conll2002 中的西班牙文資料，將 test a 作為 development set 幫助模型找到最佳參數，再使用該參數以及 training set 訓練模型，以 test set b 做驗證評估
* 將訓練 CRF 來預測 NER 結果
* CRF（Conditional Random Field）模型適合用於序列標註問題，如自然語言處理中的命名實體識別、詞性標註等。它通過學習輸入序列和輸出序列之間的條件機率分佈，並運用動態規划算法來預測給定輸入序列對應的最優輸出序列。
* 最終將列印出 precision、recall 以及 f-score 來做評估，其中包含 macro、micro 和 weighted，可以更全面地去看模型表現


#### 下載資料集

In [16]:
nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/biaoyun/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

#### 看看資料的內容

In [17]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

#### 載入其中西班牙文的 training set、test set a 和 test set b

In [19]:
# load the dataset
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
testa_sents = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
testb_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))



#### 使用 test a 做 development set 並 shuffle 該資料，確保不會因爲順序性而影響模型訓練

In [20]:
# shuffle development set

random.seed(413)
random.shuffle(train_sents)
dev_sents = testa_sents
random.shuffle(dev_sents)




#### 印出所有資料的大小確認

In [21]:
print("Train set size:", len(train_sents))
print("Development set size:", len(dev_sents))
print("Test set A size:", len(testa_sents))
print("Test set B size:", len(testb_sents))

Train set size: 8323
Development set size: 1915
Test set A size: 1915
Test set B size: 1517


#### 由於 CRF 需要自訂 features 讓模型有依據訓練（去算條件機率），所以在這邊定義 feature function

In [22]:
# extract features from the data

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]



# convert the data into features and labels
feat_train = [sent2features(s) for s in train_sents]
label_train = [sent2labels(s) for s in train_sents]

feat_dev = [sent2features(s) for s in dev_sents]
label_dev = [sent2labels(s) for s in dev_sents]

feat_test = [sent2features(s) for s in testb_sents]
label_test = [sent2labels(s) for s in testb_sents]




#### 利用 development set 以 GridSearchCV 的方式幫助模型找到最佳參數並將其印出

In [23]:
# define the parameter grid to search
params_space = {
    'c1': [0.1, 0.5, 1.0],
    'c2': [0.1, 0.5, 1.0],
    'max_iterations': [50, 100, 200],
}

# use GridSearchCV and development set to search for the best parameters
from sklearn.model_selection import GridSearchCV

crf = sklearn_crfsuite.CRF(algorithm='lbfgs', all_possible_transitions=True)
grid = GridSearchCV(crf, params_space, cv=3)
grid.fit(feat_dev , label_dev)

# print the best parameters and the corresponding score
print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)



Best parameters: {'c1': 0.1, 'c2': 0.1, 'max_iterations': 100}
Best score: 0.9631304076437125


#### 用上面找到的最佳參數，以 training set 來做最終的模型訓練

In [24]:
# train the final model with the best parameters and evaluate on the test set
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=grid.best_params_['c1'],
    c2=grid.best_params_['c2'],
    max_iterations=grid.best_params_['max_iterations'],
    all_possible_transitions=True
)

try:
    crf.fit(feat_train, label_train)
except AttributeError:
    pass



#### 刪除BIO label O 來評估 CRF 模型

* 在基於BIO（Beginning, Inside, Outside）標注方式的 NER 中，"O" 表示 "Outside"，也就是說該 token 不是命名實體的一部分。例如：

    "The company named Apple is located in California."
    
    在這個句子中，Apple 被標記為 B-ORG，表示它是一個命名實體的開始，ORG 代表它是一個組織名稱。而其他詞如 the、 is、located、 in 被標記為 "O"，是因為它們不屬於命名實體的一部分。
    
    
* 評估模型時我們著重於有 BIO 標記的部分有沒有預測正確，故刪除 "O" 標記

In [25]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-PER', 'I-MISC', 'I-ORG', 'I-LOC']

#### 模型評估

In [26]:

label_pred = crf.predict(feat_test)

# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print('Test set score:')
print(metrics.flat_classification_report(
    label_test, label_pred, labels=sorted_labels, digits=3
))

Test set score:
              precision    recall  f1-score   support

       B-LOC      0.810     0.784     0.797      1084
       I-LOC      0.690     0.637     0.662       325
      B-MISC      0.731     0.569     0.640       339
      I-MISC      0.699     0.589     0.639       557
       B-ORG      0.807     0.832     0.820      1400
       I-ORG      0.852     0.786     0.818      1104
       B-PER      0.850     0.884     0.867       735
       I-PER      0.893     0.943     0.917       634

   micro avg      0.813     0.787     0.799      6178
   macro avg      0.791     0.753     0.770      6178
weighted avg      0.809     0.787     0.796      6178



## 第三題 Reflection

這邊發現 micro 的分數比其他兩種高，可能原因如下：

1. micro metrics 是將每個預測和實際標籤作為單個二元分類問題來計算的，然後將結果進行總和。這種方法考慮了每個預測對整體指標的貢獻，因此在樣本數量不平衡的情況下，對於樣本較多的類別具有更高的權重。

2. macro metrics 是對每個類別進行單獨計算，然後取平均值。這種方法平等地考慮了每個類別的貢獻，因此在樣本數量不平衡的情況下，對於樣本較少的類別具有更高的權重。

3. weighted metrics 是對每個類別進行單獨計算，然後根據每個類別的樣本數量加權平均。這種方法同樣考慮了每個類別的貢獻，但是對於樣本數量較多的類別具有更高的權重。

因此，當一個類別的樣本數量非常大時，它對於 micro 和 weighted metrics 的影響更大，因為它對於總體結果的貢獻更大。而 macro metrics 平等地考慮了每個類別的貢獻，因此當每個類別的樣本數量大致相等時，macro metrics可能會更具代表性。

很顯然，該資料集每種標記的數量其實不是那麼平均，所以樣本較多的類別真的大部分的分數都比較高，從而讓 micro 和 weighted 的分數都高於 macro，其中，weighted 算是融合了 micro 和 macro 的特點，故最終結果仍稍微低了 micro 一些。而 macro 平等考量的方式在類別不平衡的資料中受到極端值的影響少了一些，若此時極端值得分數剛好很高，如同本次評估的結果，則最後分數反而會低於 micro 或 weighted